In [1]:
import spacy
from spacy import displacy
import random
from spacy.util import minibatch,compounding
import pandas as pd

In [2]:
nlp=spacy.load('en_core_web_sm')
nlp.pipe_names

['tagger', 'parser', 'ner']

In [3]:
ner=nlp.get_pipe('ner')

In [4]:
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("I reached Chennai yesterday.", {"entities": [(18, 28, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(31,37, "ORG")]}),
              ("I was driving a BMW", {"entities": [(16,19, "PRODUCT")]}),
              ("I ordered this from ShopClues", {"entities": [(20,29, "ORG")]}),
              ("Fridge can be ordered in Amazon ", {"entities": [(0,7, "PRODUCT")]}),
              ("I bought a new Washer", {"entities": [(16,22, "PRODUCT")]}),
              ("I bought a old table", {"entities": [(16,21, "PRODUCT")]}),
              ("I bought a fancy dress", {"entities": [(18,23, "PRODUCT")]}),
              ("I rented a camera", {"entities": [(12,18, "PRODUCT")]}),
              ("I rented a tent for our trip", {"entities": [(12,16, "PRODUCT")]}),
              ("I rented a screwdriver from our neighbour", {"entities": [(12,22, "PRODUCT")]}),
              ("I repaired my computer", {"entities": [(15,23, "PRODUCT")]}),
              ("I got my clock fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("I got my truck fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("Flipkart started it's journey from zero", {"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Max", {"entities": [(24,27, "ORG")]}),
              ("Flipkart is recognized as leader in market",{"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Swiggy", {"entities": [(24,29, "ORG")]})
              ]

In [5]:
for _,annotation in TRAIN_DATA:
  for ent in annotation.get('entities'):
    ner.add_label(ent[2])

In [6]:
pipe_want=['ner','trf_wordpiecer','trf_tok2vec']
use_pipe=[pipe for pipe in nlp.pipe_names if pipe not in pipe_want]

#Training the model

In [10]:
with nlp.disable_pipes(*use_pipe):
  for iteration in range(30):
    random.shuffle(TRAIN_DATA)
    losse={}
    batches=minibatch(TRAIN_DATA,size=compounding(4.0,32.0,1.001))
    for batch in batches:
      texts,annotation=zip(*batch)
      nlp.update(texts,
                 annotation,
                 drop=0.25,
                 losses=losse)

#Now testing our model

In [12]:
docs=nlp('Walmart is a leading e-commerce company')
for ent in docs.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)
displacy.render(nlp(docs.text),style='ent',jupyter=True)

Walmart 0 7 ORG


In [14]:
ss=nlp('I reached Chennai yesterday.')
for ent in ss.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)
displacy.render(nlp(ss.text),style='ent',jupyter=True)

yesterday 18 27 DATE
